In [2]:
%matplotlib inline
import player as data_pl
import team as data_te
import game as data_ga
import utils as anal_ut
import multiprocessing as mp
import numpy as np
import scipy
import scipy.stats as scst
import datetime
from numpy import linalg
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10.0, 8.0)
import matplotlib.colors as colors
import pandas as pd
import time
from sklearn import cluster
from sklearn import linear_model
from collections import Counter
import math
import statsmodels.api as sm
import urllib2
import re
import os
import unicodedata
import json
import pulp_lineup_optimization as lopt
from scipy import special
import cvxopt
import dfs_utils as dfsutils
DFS_SITES = dfsutils.DFS_SITES

In [32]:
with open('pid2pos.txt', 'r') as reader:
    pid2pos = eval(reader.read())
        
global_stats = pd.read_csv('global_stats.csv', index_col=0)
global_stats.GAME_DATE = [datetime.datetime.strptime(x, '%Y-%m-%d').date() for x in global_stats.GAME_DATE]
global_stats = global_stats.sort('GAME_DATE', ascending=True)

In [33]:
reload(dfsutils)
today = datetime.datetime.now().date()
pl = dfsutils.get_player_list(today)
pl

Sampson, Jakarr
Sampson, Jakarr
Sampson, Jakarr
Dunigan, Michael
Pecherov, Oleksiy
Christmas, Dionte
Crawford, Jordan
Scott, Shannon
Sims, Henry
Cooley, Jack
White, Terrico
Barron, Earl
Janning, Matt
Roberts, Ronald
Minnerath, Nick
Brown, Jabari
Sosa, Edgar
Daye, Austin
Cook, Quinn
Johnson, Nick
Casey, Kyle
Williams, Reggie
Simmons, Marcus
Sweetney, Devin
Holmes, Jonathan
Jones, DeQuan
Kyser, Michale
Stephens, D.J.
Sampson, Jakarr
Sampson, Jakarr
Series([], Name: LAST_COMMA_FIRST, dtype: object)
Sampson, Jakarr
Series([], Name: LAST_COMMA_FIRST, dtype: object)
['ATL@TOR' 'CHI@SAS' 'CLE@LAL' 'PHX@DEN']
['ATL@TOR' 'CHI@SAS' 'CLE@LAL' 'PHX@DEN']
['ATL@TOR' 'CHI@SAS' 'CLE@LAL' 'PHX@DEN']
['ATL@TOR' 'CHI@SAS' 'CLE@LAL' 'PHX@DEN']
['ATL@TOR' 'CHI@SAS' 'CLE@LAL' 'PHX@DEN']
['ATL@TOR' 'CHI@SAS' 'CLE@LAL' 'PHX@DEN']


,First Name,Last Name,Game,Team,Opponent,Injury Indicator,Injury Details,LAST_COMMA_FIRST,FD_Salary,FD_Position,...,YH_Position,Time,DP_Salary,DP_Position,DD_Salary,DD_Position,FA_Salary,FA_Position,FF_Salary,FF_Position
Player_ID,,,,,,,,,,,,,,,,,,,,,
977,Kobe,Bryant,CLE@LAL,LAL,CLE,GTD,Shoulder,"Bryant, Kobe",5700,SG,...,SF,10:30PM EST,30.23,SG,10450,"SG,SF",4600,SG,74600,F
1495,Tim,Duncan,CHI@SAS,SAS,CHI,NaN,NaN,"Duncan, Tim",5300,C,...,C,8:00PM EST,20.75,C,11300,"C,PF",4750,C,91900,C
1889,Andre,Miller,CHI@SAS,SAS,CHI,NaN,NaN,"Miller, Andre",3600,PG,...,SG,8:00PM EST,6.68,PG,5000,"PG,SG",3300,PG,35000,G
1897,Metta,World Peace,CLE@LAL,LAL,CLE,NaN,NaN,"World Peace, Metta",3500,SF,...,SF,10:30PM EST,8.75,SF,5000,"PF,SF",3300,SF,35000,F
1938,Manu,Ginobili,CHI@SAS,SAS,CHI,NaN,NaN,"Ginobili, Manu",4200,SG,...,SG,8:00PM EST,17.80,SG,7500,"SG,SF",3950,SG,60800,G
2034,Mike,Miller,PHX@DEN,DEN,PHX,NaN,NaN,"Miller, Mike",3500,SF,...,SF,9:00PM EST,5.00,SF,5000,"SG,SF",3000,SF,35000,G
2199,Tyson,Chandler,PHX@DEN,PHX,DEN,NaN,NaN,"Chandler, Tyson",4400,C,...,C,9:00PM EST,20.58,C,9050,C,4100,C,66900,C
2200,Pau,Gasol,CHI@SAS,CHI,SAS,NaN,NaN,"Gasol, Pau",9500,C,...,C,8:00PM EST,43.85,C,18750,"C,PF",6100,C,141200,C
2210,Richard,Jefferson,CLE@LAL,CLE,LAL,NaN,NaN,"Jefferson, Richard",3500,SF,...,SF,10:30PM EST,9.25,SF,5000,"SG,SF",3300,SF,35000,F


In [34]:
start_time = time.time()
filename = 'Parameters/player_parameters_%s.csv' % today
player_params = pd.read_csv(filename, index_col=False)
player_params = player_params.set_index('PERSON_ID')

with open('AdjustmentRegressions/opp_vs_pos_%s' % today, 'r') as reader:
    tvp_adj = eval(reader.read().replace('array','np.array'))

global_stats = global_stats.sort('GAME_DATE', ascending=True)
tmp = global_stats[global_stats.GAME_DATE < today]
projs = pd.DataFrame()
for row_num, row in pl.iterrows():
    if np.isnan(row.name):
        print row.LAST_COMMA_FIRST, ' has nan pid'
        continue
    pid = int(row.name)

    team_abbr = row.Team

    # Estimate Playing Time
    tmpsub = tmp[tmp.Player_ID == pid]
    if len(tmpsub) == 0:
        print row.LAST_COMMA_FIRST, 'has no data'
        continue
    if len(tmpsub.MIN.iloc[-7:])>0:
        mins = .5*(tmpsub.MIN.iloc[-7:].median() + tmpsub.MIN.iloc[-7:].mean())
        tmpsub2 = tmpsub[tmpsub.GAME_DATE >= datetime.date(2015, 7, 7)].sort('MIN')
        ngames = len(tmpsub2.MIN)
        try:
            mad = np.median(np.abs(tmpsub2.MIN - tmpsub2.MIN.median()))
            floor_mins = tmpsub2.MIN.iloc[ngames/10]
            ceil_mins  = tmpsub2.MIN.iloc[(9*ngames+1)/10]
        except:
            floor_mins = np.maximum(mins - 10, 0)
            ceil_mins = mins + 5
    else:
        print row.LAST_COMMA_FIRST
        mins = 0
        floor_mins = 0
        ceil_mins = 20

    params = player_params.loc[pid].copy()
    rates = dfsutils.params2rates(pd.DataFrame(params).T)

    try:
        adj = tvp_adj[pid2pos[pid]][row.Opponent]
        proj = dfsutils.rates_mins2projs(rates, mins, adj)
        unadj = dfsutils.rates_mins2projs(rates, mins)
        for SITE in dfsutils.DFS_SITES:
            proj['UNADJ_%s'%SITE] = unadj[SITE]
        floor_proj = dfsutils.rates_mins2projs(rates, floor_mins, adj)
        ceil_proj  = dfsutils.rates_mins2projs(rates, ceil_mins,  adj)
        proj['FLOOR_MIN'] = floor_mins
        proj['CEIL_MIN'] = ceil_mins
        for site in DFS_SITES:
            proj['FLOOR_%s'%site] = floor_proj[site]
            proj['CEIL_%s'%site] = ceil_proj[site]
        tmpsubseason = tmpsub[tmpsub.SEASON_ID==22015]
        proj['HIST_FD_Skew'] = tmpsubseason.FD.skew()
        proj['HIST_FD_Kurt'] = tmpsubseason.FD.kurt()
        minstd = np.std(tmpsubseason.MIN, ddof=1)
        if np.isnan(minstd):
            minstd = 40.
        proj['HIST_MIN_std'] = minstd
        proj['HIST_FD_std'] = np.std(tmpsubseason.FD, ddof=1)
        proj['Bust5'] = np.mean(tmpsub.FD.iloc[-5:]  <=.0035*pl.loc[pid].FD_Salary)
        proj['Bust10'] = np.mean(tmpsub.FD.iloc[-10:]<=.0035*pl.loc[pid].FD_Salary)
        proj['Bust20'] = np.mean(tmpsub.FD.iloc[-20:]<=.0035*pl.loc[pid].FD_Salary)
        proj['Boom5'] = np.mean(tmpsub.FD.iloc[-5:]  >=.005*pl.loc[pid].FD_Salary)
        proj['Boom10'] = np.mean(tmpsub.FD.iloc[-10:]>=.005*pl.loc[pid].FD_Salary)
        proj['Boom20'] = np.mean(tmpsub.FD.iloc[-20:]>=.005*pl.loc[pid].FD_Salary)
    except KeyError:
        print row.LAST_COMMA_FIRST, ' pid not in pid2pos'
        continue
    proj = pd.DataFrame(proj, columns=[pid]).T
    proj.columns = ['PROJ_%s' % x for x in proj.columns]
    proj['Player_ID'] = pid
    proj = proj.set_index('Player_ID')
    projs = projs.append(proj)

print 'Time Elapsed:', time.time() - start_time
projs

Brown, Lorenzo has no data
Pressey, Phil has no data
McRae, Jordan has no data
Williams, Alan has no data
Time Elapsed: 214.688734055


,PROJ_AST,PROJ_BLK,PROJ_FG3A,PROJ_FTA,PROJ_OREB,PROJ_DREB,PROJ_STL,PROJ_TOV,PROJ_REB,PROJ_FGM,...,PROJ_HIST_FD_Skew,PROJ_HIST_FD_Kurt,PROJ_HIST_MIN_std,PROJ_HIST_FD_std,PROJ_Bust5,PROJ_Bust10,PROJ_Bust20,PROJ_Boom5,PROJ_Boom10,PROJ_Boom20
Player_ID,,,,,,,,,,,,,,,,,,,,,
977,3.564596,0.140126,5.865304,3.857794,0.676122,2.817506,0.775830,2.052667,3.493628,5.205645,...,0.470068,0.164368,5.259911,10.326790,0.6,0.3,0.400000,0.0,0.3,0.250000
1495,2.472394,1.236091,0.037509,1.703244,1.829907,5.072099,0.577263,1.199482,6.902006,3.224672,...,0.096750,-0.221881,5.482121,9.814285,0.0,0.3,0.350000,0.4,0.3,0.350000
1889,1.889776,0.020901,0.145682,0.612915,0.290470,0.724648,0.284385,0.632569,1.015117,1.189555,...,1.251799,0.394450,7.897621,7.865187,0.8,0.9,0.950000,0.2,0.1,0.050000
1897,0.892730,0.324474,2.190859,0.960452,0.643967,1.527251,0.635078,0.498353,2.171218,1.597246,...,0.419868,-1.007784,5.879219,5.265256,1.0,0.8,0.700000,0.0,0.0,0.050000
1938,2.965121,0.179761,2.811322,1.775777,0.458211,2.103382,0.964011,1.418749,2.561593,3.504834,...,0.028522,0.980334,4.787083,8.051754,0.6,0.6,0.300000,0.4,0.3,0.400000
2034,0.449167,0.047206,0.729910,0.019515,0.074153,0.654918,0.160684,0.218493,0.729071,0.326799,...,1.080731,0.710268,5.460925,4.354663,1.0,1.0,1.000000,0.0,0.0,0.000000
2199,0.787549,0.909279,0.016236,3.147968,2.638732,6.422379,0.574626,1.387040,9.061111,2.864744,...,1.201374,2.549159,7.710143,10.211807,0.6,0.4,0.300000,0.4,0.4,0.450000
2200,3.831448,1.694287,0.711405,4.069498,2.098578,9.356122,0.564874,1.804227,11.454700,6.249468,...,0.301875,0.389090,4.646662,11.206242,0.2,0.2,0.150000,0.6,0.3,0.250000
2210,0.545272,0.143175,1.575551,0.918850,0.132485,0.999426,0.247182,0.321781,1.131911,1.332694,...,0.574662,-0.319727,8.318014,5.821698,1.0,0.8,0.750000,0.0,0.2,0.150000


In [35]:
projs.to_csv('temporary_projs.csv')
# projs = pd.read_csv('temporary_projs.csv', index_col=0)

In [38]:
projs2 = projs.copy()

overrides = {
    # LAST_COMMA_FIRST : MIN
    'Butler, Jimmy' : 0,			#CHI
    'Portis, Bobby' : 0,			#CHI
    'Bairstow, Cameron' : 0,			#CHI
    'Mozgov, Timofey' : 0,			#CLE
    'Williams, Mo' : 0,			#CLE
    'Gallinari, Danilo' : 0,			#DEN
    'Bryant, Kobe' : 0,			#LAL
    'Williams, Louis' : 0,			#LAL
    'Brown, Anthony' : 0,			#LAL
    'Parker, Tony' : 0,			#SAS
    'Carroll, DeMarre' : 0,			#TOR
}

increases = {
    # LCF : (plus, times)
}

filename = 'Parameters/player_parameters_%s.csv' % today
player_params = pd.read_csv(filename, index_col=False)
player_params = player_params.set_index('PERSON_ID')
tmp = global_stats[global_stats.GAME_DATE < today]
for lcf, mins in overrides.iteritems():
    try:
        lcf = dfsutils.COMMA_NAME_CORRECTIONS[lcf]
    except:
        pass
    pid = dfsutils.get_pid(lcf)
    if np.isnan(pid):
        print lcf, ' has nan pid'
        continue
    pid = int(pid)

    try:
        row = pl.loc[pid]
    except:
        print lcf, 'not found'
        continue

    params = player_params.loc[pid].copy()
    rates = dfsutils.params2rates(pd.DataFrame(params).T)

    try:
        proj = dfsutils.rates_mins2projs(rates, mins, tvp_adj[pid2pos[pid]][row.Opponent])
        unadj = dfsutils.rates_mins2projs(rates, mins)
        for SITE in dfsutils.DFS_SITES:
            proj['UNADJ_%s'%SITE] = unadj[SITE]
    except KeyError:
        print row.LAST_COMMA_FIRST, ' pid not in pid2pos'
        continue
    proj = pd.DataFrame(proj, columns=[pid]).T
    proj.columns = ['PROJ_%s' % x for x in proj.columns]
    proj['Player_ID'] = pid
    proj = proj.set_index('Player_ID')

    for col in proj.columns:
        projs2.set_value(pid, col, proj[col].iloc[0])

PROJS = pl.merge(projs2, left_index=True, right_index=True)

for name, s in increases.iteritems():
    f, r = s
    pid = dfsutils.get_pid(name)
    for site in DFS_SITES:
        PROJS.set_value(pid, 'PROJ_%s'%site, f + r*PROJS.loc[pid]['PROJ_%s'%site])

for SITE in dfsutils.DFS_SITES:
    PROJS['ADJ_%s'%SITE] = (PROJS['PROJ_%s'%SITE] - PROJS['PROJ_UNADJ_%s'%SITE]) / PROJS['PROJ_MIN']
PROJS['Injury Indicator'] = PROJS['Injury Indicator'].fillna('')
PROJS['Injury Details'] = PROJS['Injury Details'].fillna('')
PROJS['Status'] = [' - '.join([x,y]) if x != '' else '-' for x,y in zip(PROJS['Injury Indicator'],PROJS['Injury Details'])]

PROJS.sort('PROJ_FD', ascending=False)

,First Name,Last Name,Game,Team,Opponent,Injury Indicator,Injury Details,LAST_COMMA_FIRST,FD_Salary,FD_Position,...,PROJ_Boom10,PROJ_Boom20,ADJ_FD,ADJ_DK,ADJ_DP,ADJ_YH,ADJ_DD,ADJ_FA,ADJ_FF,Status
Player_ID,,,,,,,,,,,,,,,,,,,,,
2544,LeBron,James,CLE@LAL,CLE,LAL,,,"James, LeBron",10100,SF,...,0.3,0.300000,0.117903,0.117723,0.121639,0.119901,0.117983,0.118185,0.118185,-
200768,Kyle,Lowry,ATL@TOR,TOR,ATL,,,"Lowry, Kyle",8900,PG,...,0.4,0.300000,-0.030419,-0.024399,-0.027192,-0.033556,-0.024040,-0.030057,-0.030057,-
202695,Kawhi,Leonard,CHI@SAS,SAS,CHI,,,"Leonard, Kawhi",8700,SF,...,0.6,0.300000,0.003067,-0.005913,0.000754,-0.000389,-0.014474,0.002524,0.002524,-
2200,Pau,Gasol,CHI@SAS,CHI,SAS,,,"Gasol, Pau",9500,C,...,0.3,0.250000,-0.044474,-0.050731,-0.044398,-0.045846,-0.052912,-0.044214,-0.044214,-
200794,Paul,Millsap,ATL@TOR,ATL,TOR,,,"Millsap, Paul",8000,PF,...,0.4,0.350000,-0.046519,-0.051931,-0.049117,-0.045492,-0.048383,-0.047041,-0.047041,-
202681,Kyrie,Irving,CLE@LAL,CLE,LAL,,,"Irving, Kyrie",7200,PG,...,0.5,0.500000,0.092617,0.090206,0.093710,0.093155,0.090989,0.092674,0.092674,-
200746,LaMarcus,Aldridge,CHI@SAS,SAS,CHI,,,"Aldridge, LaMarcus",7900,PF,...,0.3,0.350000,0.015320,0.013345,0.017829,0.015272,0.007841,0.015867,0.015867,-
202688,Brandon,Knight,PHX@DEN,PHX,DEN,,,"Knight, Brandon",6500,PG,...,0.3,0.350000,0.006983,0.004134,0.008492,0.006773,0.009566,0.007137,0.007137,-
201143,Al,Horford,ATL@TOR,ATL,TOR,,,"Horford, Al",7200,C,...,0.4,0.350000,0.014431,0.006758,0.011256,0.013276,0.005565,0.013606,0.013606,-


In [39]:
PROJS.to_csv('Projections/nba_%s.csv' % today)

## Help With Minutes

In [37]:
tmp = PROJS[['O -' in x or 'D -' in x for x in PROJS.Status]].sort('Team', ascending=True)
for f,l,t in zip(tmp['First Name'], tmp['Last Name'], tmp.Team):
    print "'%s, %s' : 0,\t\t\t#%s" % (l,f,t)

'Butler, Jimmy' : 0,			#CHI
'Portis, Bobby' : 0,			#CHI
'Bairstow, Cameron' : 0,			#CHI
'Mozgov, Timofey' : 0,			#CLE
'Williams, Mo' : 0,			#CLE
'Gallinari, Danilo' : 0,			#DEN
'Bryant, Kobe' : 0,			#LAL
'Williams, Louis' : 0,			#LAL
'Brown, Anthony' : 0,			#LAL
'Parker, Tony' : 0,			#SAS
'Carroll, DeMarre' : 0,			#TOR


In [121]:
gids = np.unique(global_stats[(global_stats.Player_ID==dfsutils.get_pid('Gallinari, Danilo'))].Game_ID)
name = "Harris, Gary"

tmp = global_stats[(global_stats.Player_ID==dfsutils.get_pid(name)) & ([x  in gids for x in global_stats.Game_ID]) & (global_stats.GAME_DATE<datetime.date(2016, 12, 26)) & (global_stats.SEASON_ID==22015)]
print 'With'
print len(tmp.FD)
print tmp.FD.sum() / tmp.MIN.sum()
print tmp.MIN.median(), tmp.MIN.mean()
print tmp['USG%'].mean()

tmp = global_stats[(global_stats.Player_ID==dfsutils.get_pid(name)) & ([x not in gids for x in global_stats.Game_ID]) & (global_stats.GAME_DATE<datetime.date(2016, 12, 26)) & (global_stats.SEASON_ID==22015)]
print 'With Out'
print len(tmp.FD)
print tmp.FD.sum() / tmp.MIN.sum()
print tmp.MIN.median(), tmp.MIN.mean()
print tmp['USG%'].mean()

With
47
0.598700410397
30.0 31.1063829787
0.157171148251
With Out
6
0.640566037736
34.5 35.3333333333
0.188156502571
